In [ ]:
import pickle

from keras.models import Model,Input
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import Embedding,Dense,BatchNormalization,Dropout,LSTM,add
from keras.utils import plot_model
from keras.models import model_from_json
from keras.models import load_model

In [1]:
def load_file(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

In [ ]:
# standard variables
MAX_LENGTH = 39
VOCAB_SIZE = 1665
NPIX = 299
TARGET_SIZE = (NPIX,NPIX,3)
EMBEDDING_SIZE = 300


In [ ]:
# partial caption sequence model

inputs2 = Input(shape=(MAX_LENGTH,))
se1 = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(300)(se2)

In [ ]:
# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(300, activation='relu')(fe1)

In [ ]:
decoder1 = add([fe2, se3])
decoder2 = Dense(300, activation='relu')(decoder1)
outputs = Dense(VOCAB_SIZE, activation='softmax')(decoder2)

In [ ]:
# merge the two input models
# image_feature + partial caption ===> output
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
model.summary()

In [ ]:
embedding_matrix = load_file('train_image_extracted.pkl')

descriptions = load_file('coco_descriptions.pkl')

In [ ]:
print(model.layers[2])

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# keras library import  for Saving and loading model and weights


model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")